In [1]:
import pandas as pd
import numpy as np

In [2]:
states = pd.read_csv('states.csv')
conversion_paths = pd.read_csv('conversion_paths.csv')

# - Cleaning States Data

   * Data inspection
   * Identify and handle duplicates, null values, data types, and value types
   * Deal with null values

In [3]:
display(states.shape)
states.head(5)

(184882, 2)

,user_pseudo_id,state
0,hVFl1B9VCClNRvIkpxOUfhB9P/6a832e1Dc2ZX+YGNDCah...,North Dakota
1,KBEwoksttJ+YiA6EFk0EiR5d7LIfHYuTcD9jFLFgG/a2gr...,Maryland
2,S27mHCNkECsdX3td8QrOEplOyacPaiMNSnSiYkrPyIq7Xe...,New Jersey
3,1AkFYd/UnL/x3YVszCg6XQdvR0vrzdb37m809BpOSpqeZJ...,Nevada
4,w6IqLfvEa1J/i6mHkJvoNbL4/FDZM6hFOUmj7rekuj2sXe...,West Virginia


In [4]:
duplicates = states.duplicated(subset=['user_pseudo_id'], keep=False)
duplicates.sum()

37432

In [5]:
null_data = states.isnull().sum()
null_data

user_pseudo_id        0
state             46008
dtype: int64

In [6]:
states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184882 entries, 0 to 184881
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   user_pseudo_id  184882 non-null  object
 1   state           138874 non-null  object
dtypes: object(2)
memory usage: 2.8+ MB


In [7]:
states.describe()

,user_pseudo_id,state
count,184882,138874
unique,158504,51
top,Dlmj/GskOfHS15HDgK8nr+rDdpe2c5QgNVBm4uBlJBBi+P...,Kansas
freq,24,15389


In [8]:
#print(states['state'].value_counts())

In [9]:
states_new = states.copy()
states_new['user'] = states_new['user_pseudo_id'].str[-10:]
states_new.head()

,user_pseudo_id,state,user
0,hVFl1B9VCClNRvIkpxOUfhB9P/6a832e1Dc2ZX+YGNDCah...,North Dakota,AyzpcFfQ==
1,KBEwoksttJ+YiA6EFk0EiR5d7LIfHYuTcD9jFLFgG/a2gr...,Maryland,VogQxlgg==
2,S27mHCNkECsdX3td8QrOEplOyacPaiMNSnSiYkrPyIq7Xe...,New Jersey,LduK164g==
3,1AkFYd/UnL/x3YVszCg6XQdvR0vrzdb37m809BpOSpqeZJ...,Nevada,s73k8RsQ==
4,w6IqLfvEa1J/i6mHkJvoNbL4/FDZM6hFOUmj7rekuj2sXe...,West Virginia,JVJ7yOYQ==


In [10]:
states_new.duplicated(subset=['user_pseudo_id'], keep=False).sum()

37432

In [11]:
states_new['user'] = states_new['user'].astype('category')

In [12]:
# Group by 'user' and fill null values in 'state' with the first occurrence within each group
states_new['state_fill'] = states_new.groupby('user')['state'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

In [ ]:
states_new['state_fill'] = states_new['state']

# Itera sobre cada fila del DataFrame
for index, row in states_new.iterrows():
    # Check if the value of 'state' is null for this row
    if pd.isnull(row['state']):
        # Find other records with the same user_pseudo_id that have an assigned state
        same_user_with_state = states_new[(states_new['user'] == row['user']) & (~states_new['state'].isnull())]
        
        # If at least one record with an assigned state is found, take the first value and assign it to 'state_fill' in this row
        if len(same_user_with_state) > 0:
            new_state = same_user_with_state.iloc[0]['state']
            states_new.at[index, 'state_fill'] = new_state

In [13]:
states_new['state_fill'].fillna('Other', inplace=True)

In [14]:
states_new.isnull().sum()

user_pseudo_id        0
state             46008
user                  0
state_fill            0
dtype: int64

In [16]:
#print(states_new['state_fill'].value_counts())

In [17]:
states_new.head()

,user_pseudo_id,state,user,state_fill
0,hVFl1B9VCClNRvIkpxOUfhB9P/6a832e1Dc2ZX+YGNDCah...,North Dakota,AyzpcFfQ==,North Dakota
1,KBEwoksttJ+YiA6EFk0EiR5d7LIfHYuTcD9jFLFgG/a2gr...,Maryland,VogQxlgg==,Maryland
2,S27mHCNkECsdX3td8QrOEplOyacPaiMNSnSiYkrPyIq7Xe...,New Jersey,LduK164g==,New Jersey
3,1AkFYd/UnL/x3YVszCg6XQdvR0vrzdb37m809BpOSpqeZJ...,Nevada,s73k8RsQ==,Nevada
4,w6IqLfvEa1J/i6mHkJvoNbL4/FDZM6hFOUmj7rekuj2sXe...,West Virginia,JVJ7yOYQ==,West Virginia


In [19]:
states_new.to_csv('states_filled.csv', index=False)

# - Cleaning Conversion Path Data

 * Data inspection
 * Identify and handle duplicates, null values, data types, and value types
 * Deal with null values
 * Normalización del texto en medium path

In [4]:
display(conversion_paths.shape)
conversion_paths.head(5)

(173982, 5)

,user_pseudo_id,medium_path,conversion_flag,first_touchpoint,conversion_timestamp
0,flN8vT0gmTR1v8Ixommd1+u8s4ZvO8No7p7wl0X1v6Y+4z...,NaN,0,2022-12-28 00:00:01.914383 UTC,NaN
1,rgJvzjwn131LAGM5p0MZLCniNp8xX9VnGk861Dgu1wXNAA...,NaN,0,2022-12-28 00:00:04.792183 UTC,NaN
2,+JIUpNSpBdH11qifFKE/DwVZ5jozE6X2jJULmP3SZ0APUl...,NaN,0,2022-12-28 00:00:33.063 UTC,NaN
3,xO4dP3GBXDpOxHNwFIDCBnMKI1cBhsdZH+Eru7eyEoc0o8...,NaN,1,2022-12-28 00:00:52.439504 UTC,2023-01-23 14:16:55.060907 UTC
4,nP78ikNKi2iu1IJUO+LWIPcoByh3ZWFvlIzIl174i1jCuK...,paidsocial,0,2022-12-28 00:00:52.722434 UTC,NaN


In [5]:
conversion_paths['medium_path'] = conversion_paths['medium_path'].fillna('no medium')

In [6]:
conversion_paths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173982 entries, 0 to 173981
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   user_pseudo_id        173982 non-null  object
 1   medium_path           173982 non-null  object
 2   conversion_flag       173982 non-null  int64 
 3   first_touchpoint      173982 non-null  object
 4   conversion_timestamp  5475 non-null    object
dtypes: int64(1), object(4)
memory usage: 6.6+ MB


In [7]:
conversion_paths.duplicated(subset=['user_pseudo_id'], keep=False).sum()

0

In [8]:
conversion_paths.isnull().sum()

user_pseudo_id               0
medium_path                  0
conversion_flag              0
first_touchpoint             0
conversion_timestamp    168507
dtype: int64

In [9]:
import pytz

# Create a time zone object for the United States time zone
us_timezone = pytz.timezone('US/Eastern')

# Convert date and time columns to datetime type if they are not already
conversion_paths['first_touchpoint'] = pd.to_datetime(conversion_paths['first_touchpoint'])
conversion_paths['conversion_timestamp'] = pd.to_datetime(conversion_paths['conversion_timestamp'])

# Convert date and time columns to US time zone
conversion_paths['first_touchpoint'] = conversion_paths['first_touchpoint'].dt.tz_convert(us_timezone)
conversion_paths['conversion_timestamp'] = conversion_paths['conversion_timestamp'].dt.tz_convert(us_timezone)

conversion_paths.head()


,user_pseudo_id,medium_path,conversion_flag,first_touchpoint,conversion_timestamp
0,flN8vT0gmTR1v8Ixommd1+u8s4ZvO8No7p7wl0X1v6Y+4z...,no medium,0,2022-12-27 19:00:01.914383-05:00,NaT
1,rgJvzjwn131LAGM5p0MZLCniNp8xX9VnGk861Dgu1wXNAA...,no medium,0,2022-12-27 19:00:04.792183-05:00,NaT
2,+JIUpNSpBdH11qifFKE/DwVZ5jozE6X2jJULmP3SZ0APUl...,no medium,0,2022-12-27 19:00:33.063000-05:00,NaT
3,xO4dP3GBXDpOxHNwFIDCBnMKI1cBhsdZH+Eru7eyEoc0o8...,no medium,1,2022-12-27 19:00:52.439504-05:00,2023-01-23 09:16:55.060907-05:00
4,nP78ikNKi2iu1IJUO+LWIPcoByh3ZWFvlIzIl174i1jCuK...,paidsocial,0,2022-12-27 19:00:52.722434-05:00,NaT


In [12]:
# Convert text object columns to datetime format
conversion_paths['first_touchpoint'] = pd.to_datetime(conversion_paths['first_touchpoint'])
# Extract date and time components from 'first_touchpoint' and 'conversion_timestamp' and create new columns with datetime data type
conversion_paths['first_touchpoint_date'] = conversion_paths['first_touchpoint'].dt.date.astype('datetime64')
conversion_paths['first_touchpoint_time'] = conversion_paths['first_touchpoint'].dt.strftime('%H:%M')

#conversion_paths['first_touchpoint_date']
#conversion_paths['first_touchpoint_time']

In [13]:
# Convert text object columns to datetime format
conversion_paths['conversion_timestamp'] = pd.to_datetime(conversion_paths['conversion_timestamp'])

# Extract date and time components from 'first_touchpoint' and 'conversion_timestamp' and create new columns with datetime data type
conversion_paths['conversion_date'] = conversion_paths['conversion_timestamp'].dt.date.astype('datetime64')
conversion_paths['conversion_time'] = conversion_paths['conversion_timestamp'].dt.strftime('%H:%M')

conversion_paths['conversion_date']
#conversion_paths['conversion_time']

0               NaT
1               NaT
2               NaT
3        2023-01-23
4               NaT
            ...    
173977          NaT
173978          NaT
173979          NaT
173980          NaT
173981          NaT
Name: conversion_date, Length: 173982, dtype: datetime64[ns]

In [14]:
# Standardize texts in Medium Path
conversion_paths['medium_path'] = conversion_paths['medium_path'].str.lower()  
conversion_paths['medium_path'] = conversion_paths['medium_path'].str.strip()

In [15]:
# Validate the dataset for completeness and accuracy
conversion_paths.head()

,user_pseudo_id,medium_path,conversion_flag,first_touchpoint,conversion_timestamp,first_touchpoint_date,first_touchpoint_time,conversion_date,conversion_time
0,flN8vT0gmTR1v8Ixommd1+u8s4ZvO8No7p7wl0X1v6Y+4z...,no medium,0,2022-12-27 19:00:01.914383-05:00,NaT,2022-12-27,19:00,NaT,NaN
1,rgJvzjwn131LAGM5p0MZLCniNp8xX9VnGk861Dgu1wXNAA...,no medium,0,2022-12-27 19:00:04.792183-05:00,NaT,2022-12-27,19:00,NaT,NaN
2,+JIUpNSpBdH11qifFKE/DwVZ5jozE6X2jJULmP3SZ0APUl...,no medium,0,2022-12-27 19:00:33.063000-05:00,NaT,2022-12-27,19:00,NaT,NaN
3,xO4dP3GBXDpOxHNwFIDCBnMKI1cBhsdZH+Eru7eyEoc0o8...,no medium,1,2022-12-27 19:00:52.439504-05:00,2023-01-23 09:16:55.060907-05:00,2022-12-27,19:00,2023-01-23,09:16
4,nP78ikNKi2iu1IJUO+LWIPcoByh3ZWFvlIzIl174i1jCuK...,paidsocial,0,2022-12-27 19:00:52.722434-05:00,NaT,2022-12-27,19:00,NaT,NaN


In [15]:
# Replace null values with a 'no conversion' flag
#conversion_paths['conversion_date'].fillna('no conversion', inplace=True)
#conversion_paths['conversion_time'].fillna('no conversion', inplace=True)

In [16]:
#Drop the columns 
conversion_paths.drop(columns=['first_touchpoint', 'conversion_timestamp'], inplace=True)

In [17]:
conversion_paths

,user_pseudo_id,medium_path,conversion_flag,first_touchpoint_date,first_touchpoint_time,conversion_date,conversion_time
0,flN8vT0gmTR1v8Ixommd1+u8s4ZvO8No7p7wl0X1v6Y+4z...,no medium,0,2022-12-27,19:00,NaT,NaN
1,rgJvzjwn131LAGM5p0MZLCniNp8xX9VnGk861Dgu1wXNAA...,no medium,0,2022-12-27,19:00,NaT,NaN
2,+JIUpNSpBdH11qifFKE/DwVZ5jozE6X2jJULmP3SZ0APUl...,no medium,0,2022-12-27,19:00,NaT,NaN
3,xO4dP3GBXDpOxHNwFIDCBnMKI1cBhsdZH+Eru7eyEoc0o8...,no medium,1,2022-12-27,19:00,2023-01-23,09:16
4,nP78ikNKi2iu1IJUO+LWIPcoByh3ZWFvlIzIl174i1jCuK...,paidsocial,0,2022-12-27,19:00,NaT,NaN
...,...,...,...,...,...,...,...
173977,Bhb3oQm/4lEdnfspisZFFIQ7KsURX1kTn0ZoORLfQACaMp...,paidsocial,0,2023-01-26,18:57,NaT,NaN
173978,/Tlhkd3MP+sLk0BCYysi+42pTI0n27hWhFOoYDhu9RbyFH...,organic,0,2023-01-26,18:58,NaT,NaN
173979,RqRHrHy9ulJb0kzPZ6p3yNYs2X0GQrvwONXmEbp+1mWyZw...,no medium,0,2023-01-26,18:58,NaT,NaN
173980,2S+bgXa7/MX7Wd9i1glLSWz8tsc9AtojUmymF9GWXGOvQj...,no medium,0,2023-01-26,18:59,NaT,NaN


#    - Create a new dataset with the filtered data

* Merge both datasets for deeper analysis
* Ensure data integrity and consistency of formats
* Convert state data to numeric for potential machine learning analysis

In [18]:
states_new = pd.read_csv('states_filled.csv')

In [19]:
states_new.drop(columns=['user', 'state'], inplace=True)

In [20]:
states_new.head()

,user_pseudo_id,state_fill
0,hVFl1B9VCClNRvIkpxOUfhB9P/6a832e1Dc2ZX+YGNDCah...,North Dakota
1,KBEwoksttJ+YiA6EFk0EiR5d7LIfHYuTcD9jFLFgG/a2gr...,Maryland
2,S27mHCNkECsdX3td8QrOEplOyacPaiMNSnSiYkrPyIq7Xe...,New Jersey
3,1AkFYd/UnL/x3YVszCg6XQdvR0vrzdb37m809BpOSpqeZJ...,Nevada
4,w6IqLfvEa1J/i6mHkJvoNbL4/FDZM6hFOUmj7rekuj2sXe...,West Virginia


In [21]:
merged_data = pd.merge(states_new, conversion_paths, on='user_pseudo_id', how='inner')
merged_data

,user_pseudo_id,state_fill,medium_path,conversion_flag,first_touchpoint_date,first_touchpoint_time,conversion_date,conversion_time
0,hVFl1B9VCClNRvIkpxOUfhB9P/6a832e1Dc2ZX+YGNDCah...,North Dakota,no medium,0,2023-01-21,10:33,NaT,NaN
1,KBEwoksttJ+YiA6EFk0EiR5d7LIfHYuTcD9jFLFgG/a2gr...,Maryland,no medium,0,2023-01-23,22:53,NaT,NaN
2,S27mHCNkECsdX3td8QrOEplOyacPaiMNSnSiYkrPyIq7Xe...,New Jersey,no medium,0,2023-01-24,11:39,NaT,NaN
3,1AkFYd/UnL/x3YVszCg6XQdvR0vrzdb37m809BpOSpqeZJ...,Nevada,no medium,0,2023-01-24,15:03,NaT,NaN
4,w6IqLfvEa1J/i6mHkJvoNbL4/FDZM6hFOUmj7rekuj2sXe...,West Virginia,no medium,0,2023-01-05,04:58,NaT,NaN
...,...,...,...,...,...,...,...,...
184877,qreOzdSFp+uytQaT9vPD+emuUBbIugaxi8AE048vAQ+48Q...,South Carolina,cpc,0,2023-01-20,13:39,NaT,NaN
184878,/EtQfSZBPW5w8baapcbFC+rFPefq4JYJz29tHmtCKgfL47...,South Carolina,paidsocial,0,2023-01-20,17:34,NaT,NaN
184879,ks52NCYBZCIGPv/eoZvqidovKRoG00jj9aSsk8dlwsWlG3...,South Carolina,referral,0,2023-01-11,21:38,NaT,NaN
184880,rDwxO5UYceE/2fjGg90+KbwcfRkIvt8EBi8Si6dXHL5iED...,South Carolina,paidsocial,0,2023-01-12,13:09,NaT,NaN


In [22]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184882 entries, 0 to 184881
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   user_pseudo_id         184882 non-null  object        
 1   state_fill             184882 non-null  object        
 2   medium_path            184882 non-null  object        
 3   conversion_flag        184882 non-null  int64         
 4   first_touchpoint_date  184882 non-null  datetime64[ns]
 5   first_touchpoint_time  184882 non-null  object        
 6   conversion_date        12926 non-null   datetime64[ns]
 7   conversion_time        12926 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 12.7+ MB


In [23]:
merged_data.isnull().sum()

user_pseudo_id                0
state_fill                    0
medium_path                   0
conversion_flag               0
first_touchpoint_date         0
first_touchpoint_time         0
conversion_date          171956
conversion_time          171956
dtype: int64

In [24]:
merged_data.nunique()

user_pseudo_id           158504
state_fill                   52
medium_path                1709
conversion_flag               2
first_touchpoint_date        31
first_touchpoint_time      1440
conversion_date              31
conversion_time            1381
dtype: int64

* NOTE: List of states in EE.UU.
1	Alabama
2	Alaska
3	Arizona
4	Arkansas
5	California
6	Colorado
7	Connecticut
8	Delaware
9	Florida
10	Georgia
11	Hawaii
12	Idaho
13	Illinois
14	Indiana
15	Iowa
16	Kansas
17	Kentucky
18	Louisiana
19	Maine
20	Maryland
21	Massachusetts
22	Michigan
23	Minnesota
24	Mississippi
25	Missouri
26	Montana
27	Nebraska
28	Nevada
29	New Hampshire
30	New Jersey
31	New Mexico
32	New York
33	North Carolina
34	North Dakota
35	Ohio
36	Oklahoma
37	Oregon
38	Pennsylvania
39	Rhode Island
40	South Carolina
41	South Dakota
42	Tennessee
43	Texas
44	Utah
45	Vermont
46	Virginia
47	Washington
48	West Virginia
49	Wisconsin
50	Wyoming


In [25]:
us_states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida',
    'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
    'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
    'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
    'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
    'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'
]

# Create a dictionary that maps state names to state numbers
state_mapping = {state: index + 1 for index, state in enumerate(us_states)}

# Assign the value 0 to the "Other" category
state_mapping['Other'] = 0

# Apply the mapping to the DataFrame
merged_data['state_fill_numeric'] = merged_data['state_fill'].map(state_mapping)

# Convert column 'state_fill_numeric' to categorical data type
merged_data['state_fill_numeric'] = merged_data['state_fill_numeric'].astype('category')


In [26]:
# Validate the dataset for completeness and accuracy
merged_data.head()

,user_pseudo_id,state_fill,medium_path,conversion_flag,first_touchpoint_date,first_touchpoint_time,conversion_date,conversion_time,state_fill_numeric
0,hVFl1B9VCClNRvIkpxOUfhB9P/6a832e1Dc2ZX+YGNDCah...,North Dakota,no medium,0,2023-01-21,10:33,NaT,NaN,34.0
1,KBEwoksttJ+YiA6EFk0EiR5d7LIfHYuTcD9jFLFgG/a2gr...,Maryland,no medium,0,2023-01-23,22:53,NaT,NaN,20.0
2,S27mHCNkECsdX3td8QrOEplOyacPaiMNSnSiYkrPyIq7Xe...,New Jersey,no medium,0,2023-01-24,11:39,NaT,NaN,30.0
3,1AkFYd/UnL/x3YVszCg6XQdvR0vrzdb37m809BpOSpqeZJ...,Nevada,no medium,0,2023-01-24,15:03,NaT,NaN,28.0
4,w6IqLfvEa1J/i6mHkJvoNbL4/FDZM6hFOUmj7rekuj2sXe...,West Virginia,no medium,0,2023-01-05,04:58,NaT,NaN,48.0


In [28]:
merged_data.to_csv('dataset_new.csv', index=False)